In [1]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
# from myUtils import symb_perf_stats_vectorized_v2
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks
verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [2]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)

In [3]:
df_train, df_val, df_test = _2_split_train_val_test(df_c)

In [4]:
len(df_train)
# df_val
# df_test

1050

In [5]:
n_samples = 1
days_lookback = 60
days_eval = 10

In [6]:
my_slices = _3_random_slices(len(df_train), n_samples=n_samples, days_lookback=days_lookback, days_eval=days_eval)

In [7]:
print(my_slices[0], my_slices[0][0], my_slices[0][1], my_slices[0][2])

(660, 720, 730) 660 720 730


In [8]:
my_slices

[(660, 720, 730)]

In [9]:
# for my_slice in my_slices[0:2]:
for my_slice in my_slices:
  start_train = my_slice[0]
  end_train = my_slice[1]
  start_val = end_train
  end_val = my_slice[2]
  print(f'start_train: {start_train}')
  print(f'end_train: {end_train}')
  print(f'start_val: {start_val}')
  print(f'end_val: {end_val}')

  # _df = df_train.iloc[start_train:end_train].copy()
  _df = df_train.iloc[start_train:end_train]
  print(_df) 
  # perf_ranks_dict, ranked_perf_ranks_dict = _4_perf_ranks(_df, days_lookbacks=[60])
  # print(perf_ranks_dict)
  # print(ranked_perf_ranks_dict)  

start_train: 660
end_train: 720
start_val: 720
end_val: 730
                    A         AA        AAL       AAON         AAP       AAPL  \
Date                                                                            
2019-08-23  67.982170  17.313721  25.244576  44.613300  127.270393  49.481907   
2019-08-26  68.070145  17.006668  25.016165  45.435478  127.223259  50.422031   
2019-08-27  67.913757  16.570854  24.281271  44.789474  127.119591  49.853073   
2019-08-28  68.363380  16.848194  25.155199  45.778057  128.881866  50.187607   
2019-08-29  69.106247  17.630678  26.138367  46.521938  130.116425  51.037365   
2019-08-30  69.507004  17.759443  26.128435  46.952606  130.003296  50.971443   
2019-09-03  68.822800  17.442484  26.327055  45.151638  132.274475  50.229118   
2019-09-04  69.262642  18.106113  26.813673  46.610035  133.716354  51.081326   
2019-09-05  71.862686  19.433367  27.498913  47.393063  138.107910  52.080044   
2019-09-06  72.605545  18.928215  27.598223  46.4

In [10]:
perf_ranks_dict, ranked_perf_ranks_dict = _4_perf_ranks(_df, days_lookbacks=[60, 120])

days_lookback: -60
_df_c:                     A         AA        AAL       AAON         AAP       AAPL  \
Date                                                                            
2019-08-23  67.982170  17.313721  25.244576  44.613300  127.270393  49.481907   
2019-08-26  68.070145  17.006668  25.016165  45.435478  127.223259  50.422031   
2019-08-27  67.913757  16.570854  24.281271  44.789474  127.119591  49.853073   
2019-08-28  68.363380  16.848194  25.155199  45.778057  128.881866  50.187607   
2019-08-29  69.106247  17.630678  26.138367  46.521938  130.116425  51.037365   
2019-08-30  69.507004  17.759443  26.128435  46.952606  130.003296  50.971443   
2019-09-03  68.822800  17.442484  26.327055  45.151638  132.274475  50.229118   
2019-09-04  69.262642  18.106113  26.813673  46.610035  133.716354  51.081326   
2019-09-05  71.862686  19.433367  27.498913  47.393063  138.107910  52.080044   
2019-09-06  72.605545  18.928215  27.598223  46.492577  138.729889  52.075165   
20

In [11]:
# print(perf_ranks_dict)
trash = perf_ranks_dict
trash

{'period-60': {'r_CAGR/UI': array(['RETA', 'HOV', 'TNK', ..., 'CTRE', 'NRP', 'MITK'], dtype=object),
  'r_CAGR/Std': array(['HOV', 'RETA', 'TNK', ..., 'DENN', 'PSA', 'NXDT'], dtype=object),
  'r_Std/UI': array(['UI', 'SNX', 'SSD', ..., 'OR', 'PRDO', 'LOPE'], dtype=object)},
 'period-120': {'r_CAGR/UI': array(['RETA', 'HOV', 'TNK', ..., 'CTRE', 'NRP', 'MITK'], dtype=object),
  'r_CAGR/Std': array(['HOV', 'RETA', 'TNK', ..., 'DENN', 'PSA', 'NXDT'], dtype=object),
  'r_Std/UI': array(['UI', 'SNX', 'SSD', ..., 'OR', 'PRDO', 'LOPE'], dtype=object)}}

In [12]:
ranked_perf_ranks_dict

{'ranked_perf_ranks_period[60, 120]': ['RETA',
  'HOV',
  'TNK',
  'ISEE',
  'GPRE',
  'UI',
  'SYNA',
  'ODP',
  'TSLA',
  'CNXN',
  'HIBB',
  'UCTT',
  'SNX',
  'DOOR',
  'SSD',
  'COHU',
  'BMY',
  'VSTO',
  'KRG',
  'JBL',
  'SCVL',
  'ABBV',
  'ALGN',
  'TSM',
  'AAPL',
  'PBF',
  'FORM',
  'NSIT',
  'SHOO',
  'BHE',
  'INSW',
  'VLO',
  'DEA',
  'NVMI',
  'SHYF',
  'KOP',
  'AMWD',
  'BLDR',
  'SP',
  'QRVO',
  'TILE',
  'HNI',
  'STM',
  'IT',
  'FTSM',
  'CROX',
  'OEC',
  'HOFT',
  'SBH',
  'RMAX',
  'THO',
  'GAIN',
  'ALLE',
  'BRX',
  'CHT',
  'MBUU',
  'FIX',
  'LRCX',
  'SKX',
  'AEHR',
  'TAK',
  'LNW',
  'MPC',
  'ICHR',
  'CAL',
  'HEES',
  'CM',
  'STT',
  'MKSI',
  'AMAT',
  'BEP',
  'AZTA',
  'LEG',
  'SGRY',
  'KLAC',
  'GT',
  'EMR',
  'INGN',
  'PZZA',
  'GPK',
  'ABR',
  'PNC',
  'TSE',
  'LOCO',
  'OMAB',
  'PLUG',
  'WSBF',
  'JHX',
  'WGO',
  'ACLS',
  'LYB',
  'ASML',
  'BXMX',
  'TER',
  'PDFS',
  'UMH',
  'SGEN',
  'CNS',
  'ABCB',
  'SRCL',
  'LPG',
  'DC